In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scipy.io
import scanpy.external as scex
import sklearn.metrics
import matplotlib
import bbknn
#import dca
#import scvelo as scv
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.mixture import GaussianMixture as GMM
from scipy.stats import norm

In [ ]:
#adata=sc.read(results_file_post)
#adata.uns['log1p'] = {"base":None}

Set up out properties

In [ ]:
sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_header()
sc.settings.set_figure_params(dpi=80, facecolor='white')

In [ ]:
#%matplotlib inline

Set scanpy out-files

In [ ]:
writeDir = "write/"

fileName = "scSarcChemo_V4"

resultsFile = writeDir + fileName + '.h5ad'       # final output
resultsFileQC = writeDir + fileName + '_QC.h5ad'  # post QC (pre-analysis) 
#resultsFileTry = writeDir + fileName + '_fTry.h5ad'

Set figure parameters

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')
sc.settings.figdir = "figures/" + fileName + "/"
figName = fileName

read input file

In [ ]:
inputFile = 'data/sarcChemo/filtered_feature_bc_matrix.h5'

In [ ]:
adata =  sc.read_10x_h5(inputFile, gex_only=False)
adata

In [ ]:
adata.var_names_make_unique()

In [ ]:
adata.var

In [ ]:
adata.var[["feature_types"]].value_counts()

Split up input into genes and hashes 

In [ ]:
hto = adata[:,adata.var["feature_types"] == "Antibody Capture"]
adata = adata[:,adata.var["feature_types"] == "Gene Expression"]
adata.obs = pd.DataFrame(hto.X.todense(), columns=hto.var_names, index=adata.obs.index)

In [ ]:
adata.obs

# Start QC
investigate highest expressed genes

In [ ]:
sc.pl.highest_expr_genes(adata, n_top=20, )

In [ ]:
sc.pp.filter_cells(adata, min_genes = 200)
sc.pp.filter_genes(adata, min_cells = 10)
adata

## Mito QC
set genes that start with mt- as mito genes

In [ ]:
adata.var['mt'] = adata.var_names.str.startswith('mt-')  # annotate the group of mitochondrial genes as 'mt'
sc.pp.calculate_qc_metrics(adata, qc_vars=['mt'], log1p = False, inplace=True)

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
sc.pl.scatter(adata, x='total_counts', y='pct_counts_mt')
sc.pl.scatter(adata, x='total_counts', y='n_genes_by_counts')

remove cells that have too much mito or could be doublets

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 7000, :]
#adata = adata[adata.obs.n_genes_by_counts > 1000, :]
adata = adata[adata.obs.total_counts < 40000, :]
adata = adata[adata.obs.pct_counts_mt < 5, :]

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt'], jitter=0.4, multi_panel=True)

In [ ]:
#adata = sc.read(resultsFileQC)
print(resultsFileQC)
adata.write(resultsFileQC)
adata

# Perform Demultiplex Hashing

In [ ]:
hashCounts = adata.obs[hto.var_names]
hashCounts

In [ ]:
hashDisc = hashCounts.describe([.1,.2,.3,.4,.5,.6,.7,.8,.9])
hashDisc

In [ ]:
hashCounts.columns.values

In [ ]:
fig, axs = plt.subplots(2,5)
dfHashBoundry = pd.DataFrame(np.zeros(len(hto.var_names)),hto.var_names, columns=["boundry"])
gmm = GMM(n_components = 2, covariance_type = 'full', n_init=5,  random_state=10, means_init=[[2],[4]])#
#binEx = np.arange(0.5,10,10/200).reshape(-1,1)

for i, hashName in enumerate(hto.var_names):
    hashCount = np.array(np.log10(adata.obs[hashName]+1)).reshape(-1, 1)
    fitGMM = gmm.fit(hashCount)
    mean = fitGMM.means_  
    covs  = fitGMM.covariances_
    weights = fitGMM.weights_
    #print(mean)
    binEx = np.arange(min(mean),max(mean),0.1).reshape(-1,1)
    fitGmmBound = fitGMM.predict(binEx)
    #print(fitGmmBound)
    hashBoundry = binEx[np.where(fitGmmBound == 1)[0][0]][0]
    #naiveBoundry = np.log10(int(hashDisc.loc["90%",hashName])+1)
    
    dfHashBoundry.loc[hashName] = hashBoundry
    
    x_axis = np.arange(0, 5, 0.1)
    y_axis0 = norm.pdf(x_axis, float(mean[0][0]), np.sqrt(float(covs[0][0][0])))*weights[0] # 1st gaussian
    y_axis1 = norm.pdf(x_axis, float(mean[1][0]), np.sqrt(float(covs[1][0][0])))*weights[1] # 2nd gaussian

    # Plot 2
    axs[i//5,i%5].set_title(hashName)
    #axs[i//5,i%5].axvline(naiveBoundry, c='C3', linestyle='dashed', linewidth=1) #red
    axs[i//5,i%5].axvline(hashBoundry, c='C2', linestyle='dashed', linewidth=1)  #green
    axs[i//5,i%5].hist(hashCount, density=True, color='black', bins=100)        
    axs[i//5,i%5].plot(x_axis, y_axis0, lw=3, c='C6')                            #pink
    axs[i//5,i%5].plot(x_axis, y_axis1, lw=3, c='C1')                            #orange
    axs[i//5,i%5].plot(x_axis, y_axis0+y_axis1, lw=3, c='C0', ls=':')            #dotted blue
    
plt.tight_layout(pad=1.0)
plt.rcParams["figure.figsize"] = (20,5)
plt.show()

In [ ]:
hashIDs = hashCounts.copy()
hashID = hto.var_names
for hashName in hto.var_names:
    print(hashName)
    print(dfHashBoundry.loc[hashName].values[0])
    hashIDs.loc[:,hashName] = np.log10(hashCounts.loc[:,hashName]+1) > dfHashBoundry.loc[hashName].values[0]
hashIDs

In [ ]:
hashIDs.sum(axis=0)

In [ ]:
hashIDs.sum(axis=1).value_counts()

In [ ]:
classification = np.empty(len(adata), dtype="object")
i = 0
for cellBar, hashBool in hashIDs.iterrows():
    numHashes = sum(hashBool)
    if (numHashes == 1):
        classif = hashID[hashBool.values].values[0]
    elif (numHashes > 1):
        classif = "doublet"
    else:
        classif = "negative"
    classification[i] = classif
    i = i + 1
    #break

visualize hashes

count each hash

In [ ]:
adata.obs["Classification"] = classification
adata.obs["Classification"].value_counts()

In [ ]:
#output visulaization of hashing
sc.pl.heatmap(adata, hto.var_names, groupby="Classification", log=True)#, save = f"_{figName}_hash.png")

keep singlets only

In [ ]:
singlets = [x in hto.var_names for x in adata.obs["Classification"] ]
adata = adata[singlets,]
adata

label hashes

In [ ]:
treat =  {  "AV1354_MR_B0301": "wk4",
            "AVU1779_MR_B0302":"wk4",
            "AU1782_B0303":    "wk4",
            "AS1131_ML_B0304": "wk1",
            "AU1931_MR_B0305": "wk1",
            "AS1479_MR_B0306": "wk1",
            "AV1353_B0307":    "ctl",
            "AT1885_B0308":    "ctl",
            "AV1212_MR_B0309": "ctl",
            "AU1931_ML_B0310": "wk1"
}
adata.obs["treat"] = [treat[x] for x in adata.obs["Classification"]]

save post QC scanpy

In [ ]:
resultsFileQC

In [ ]:
adata = adata[:,[x != "Malat1" for x in adata.var_names]]
adata

In [ ]:
adata.var

In [ ]:
resultsFileQC

In [ ]:
adata = sc.read(resultsFileQC)

In [ ]:
#adata = sc.read(resultsFileQC)
adata.layers["counts"] = adata.X.copy()
adata.write(resultsFileQC)
adata

In [ ]:
#adata.var[adata.var["pct_dropout_by_counts"]<1]

In [ ]:
#adata = adata[:,np.logical_and(adata.var["pct_dropout_by_counts"]>10, adata.var["pct_dropout_by_counts"]<99)]

save post QC scanpy

In [ ]:
sc.set_figure_params(scanpy=True, dpi=100, dpi_save=150, fontsize=10, format='png')

In [ ]:
#x = np.array(adata.X.todense()).flatten()
#plt.hist(x[x>4], bins=50)
#plt.title('Gene dispersions counts')
#plt.xlabel('dispersions')
#plt.ylabel('counts')
#plt.show()

In [ ]:
sc.pp.normalize_total(adata, target_sum=1e4)#,exclude_highly_expressed=True)#

In [ ]:
sc.pp.log1p(adata)

In [ ]:
#x = np.array(adata.X.todense()).flatten()
#plt.hist(x[x>0], bins=20)
#plt.title('Gene dispersions counts')
#plt.xlabel('dispersions')
#plt.ylabel('counts')
#plt.show()

In [ ]:
cell_cycle_genes = [x.strip() for x in open('../pdx/data/regev_lab_cell_cycle_genes_Mouse.txt')]
s_genes = cell_cycle_genes[:43]
g2m_genes = cell_cycle_genes[43:]
cell_cycle_genes = [x for x in cell_cycle_genes if x in adata.var_names]

In [ ]:
sc.tl.score_genes_cell_cycle(adata, s_genes=s_genes, g2m_genes=g2m_genes)

In [ ]:
adata_cc_genes = adata[:, cell_cycle_genes]
sc.tl.pca(adata_cc_genes)
sc.pl.pca_scatter(adata_cc_genes, color=['phase',"Classification"])

In [ ]:
adata = adata[adata.obs.phase=="G1",:]

In [ ]:
sc.pp.highly_variable_genes(adata)#, flavor="seurat", n_top_genes=2000)

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
x = adata.var[["means"]][adata.var[["means"]] > np.exp(-20)]#adata.var[["means"]]
plt.hist(np.log(x), bins=100)#, log=True)
plt.axvline(np.log(0.05), color='k', linestyle='dashed', linewidth=1)
plt.axvline(np.log(3), color='k', linestyle='dashed', linewidth=1)
plt.title('Gene means counts')
plt.xlabel('means')
plt.ylabel('counts')
plt.show()

In [ ]:
x = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-10)]#adata.var[["dispersions"]]
plt.hist(np.log(x), bins=50)#, log=True)
plt.axvline(np.log(0.5), color='k', linestyle='dashed', linewidth=1)
plt.title('Gene dispersions counts')
plt.xlabel('dispersions')
plt.ylabel('counts')
plt.show()

In [ ]:
sc.pp.highly_variable_genes(adata, min_disp=0.5, min_mean=0.05, max_mean=3)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
adata.var[-10:]

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
adata.raw = adata

In [ ]:
#adata.var[adata.var.pct_dropout_by_counts<10].highly_variable

In [ ]:
#adata = adata[:, adata.var.highly_variable]
adata = adata[:, np.logical_and(np.logical_not(adata.var.mt), adata.var.highly_variable)]

In [ ]:
#sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
#x = np.array(adata.X.todense()).flatten()
#plt.hist(x[x>0], bins=20)
#plt.title('Gene dispersions counts')
#plt.xlabel('dispersions')
#plt.ylabel('counts')
#plt.show()

In [ ]:
#sc.pp.scale(adata, max_value=4)

In [ ]:
#x = np.array(adata.X).flatten()
#plt.hist(x[x>0], bins=20)
#plt.title('Gene dispersions counts')
#plt.xlabel('dispersions')
#plt.ylabel('counts')
#plt.show()

In [ ]:
sc.tl.pca(adata, n_comps = 100, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color = ["treat","Classification"])

In [ ]:
sc.pl.pca(adata, color = ["Classification"], dimensions=[(0,1), (1,2), (2,3),(3,1)])

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100, log=True)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100)

In [ ]:
#adata.write(results_file)
adata

In [ ]:
sc.pp.neighbors(adata, n_neighbors=20, n_pcs=50)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution=0.1)

In [ ]:
sc.pl.umap(adata, color=["treat","leiden","Classification","phase"])

In [ ]:
sc.pl.umap(adata[adata.obs.treat=="ctl",:], color=["treat","leiden","Classification"])

In [ ]:
sc.pl.umap(adata[adata.obs.treat=="1wk",:], color=["treat","leiden","Classification"])

In [ ]:
sc.pl.umap(adata[adata.obs.treat=="4wk",:], color=["treat","leiden","Classification"])

In [ ]:
sc.pl.umap(adata, color=['tdTomato','Ptprc','Lyz2', "Pecam1"],ncols=2)#, save = f"_{figName}_tumGenes.png")

In [ ]:
sc.pl.violin(adata, ["tdTomato","Ptprc","Pecam1"], jitter=0.4, multi_panel=True, groupby="leiden")

In [ ]:
sc.pl.umap(adata, color=["Acta2","Myod1","Peg3","Des"],ncols = 2)#, save = f"_{figName}_muscGenes.png")

In [ ]:
sc.pl.umap(adata, color=["Fbn1","Lum","Meg3","Postn"],ncols = 2)#, save = f"_{figName}_fibGenes.png")

In [ ]:
sc.pl.umap(adata, color=["Klf9"],ncols = 2)#, save = f"_{figName}_fibGenes.png")

In [ ]:
sc.pl.umap(adata, color=["total_counts_mt","total_counts"])

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
#as heatmap
markerDf = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(15)
markerDf

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_heatmap(adata)

In [ ]:
sc.pl.rank_genes_groups_matrixplot(adata)

In [ ]:
markerDict = {}
#[markerDict[int(i)] = markerDf.loc[0:3][i] ]
for i in markerDf:
    markerDict[int(i)] = markerDf.loc[0:3][i] 
markerDict

In [ ]:
sc.pl.umap(adata, color=["leiden"],legend_loc="on data")#, save = f"_{figName}_tumClassWk.png")

In [ ]:
sc.pl.umap(adata, color=["Krt18","Krt8","Akap9"],ncols = 2)#, save = f"_{figName}_fibGenes.png")

In [ ]:
adata.var.sort_values("pct_dropout_by_counts")[0:20]

In [ ]:
adata.var.sort_values("pct_dropout_by_counts", ascending=False)[0:20]

In [ ]:
adata.obs["Tum"] = ["tum" if x in ["0","1","2","4"] else "imm" for x in adata.obs["leiden"]]

In [ ]:
sc.pl.umap(adata, color=["treat","leiden","Tum","Classification"], ncols=2)

In [ ]:
sc.pl.umap(adata, color=["Car9","Vegfa"], ncols=2)

In [ ]:
#adata.write
adata = sc.read("write/scSarcChemo_G1.h5ad")

In [ ]:
#print(resultsFile)
#adata.write(resultsFile)
adata

In [ ]:
adataQC = sc.read("write/scSarcChemo_V4_QC.h5ad")
cells = set(adata.obs_names)
keepCells = [c in cells for c in adataQC.obs_names]
adataQC = adataQC[keepCells]

counts = adataQC.layers["counts"]

In [ ]:
#adata = sc.read(resultsFile)
#adata.uns['log1p'] = {"base":None}

In [ ]:
adata = adata.raw.to_adata()
adata.layers["counts"] = counts
adata = adata[adata.obs.Tum =="tum",:]
adata

In [ ]:
adata.uns['log1p'] = {"base":None}

In [ ]:
sc.pp.highly_variable_genes(adata)

In [ ]:
x = adata.var[["means"]][adata.var[["means"]] > np.exp(-10)]#adata.var[["means"]]
plt.hist(np.log(x), bins=100)#, log=True)
plt.axvline(np.log(0.05), color='k', linestyle='dashed', linewidth=1)
plt.axvline(np.log(2.9), color='k', linestyle='dashed', linewidth=1)
plt.title('Gene means counts')
plt.xlabel('means')
plt.ylabel('counts')
plt.show()

In [ ]:
x = adata.var[["dispersions_norm"]][adata.var[["dispersions_norm"]] > np.exp(-5)]#adata.var[["dispersions"]]
plt.hist(np.log(x), bins=50)#, log=True)
plt.axvline(np.log(0.6), color='k', linestyle='dashed', linewidth=1)
plt.title('Gene dispersions counts')
plt.xlabel('dispersions')
plt.ylabel('counts')
plt.show()

In [ ]:
sc.pp.highly_variable_genes(adata, min_disp=0.6, min_mean=0.05, max_mean=2.9)

In [ ]:
sc.pl.highly_variable_genes(adata)

In [ ]:
sum(adata.var.highly_variable)

In [ ]:
adata.raw = adata

In [ ]:
sum(np.logical_and(adata.var.highly_variable,np.logical_not(adata.var.mt)))

In [ ]:
adata = adata[:, np.logical_and(adata.var.highly_variable,np.logical_not(adata.var.mt))]

In [ ]:
#sc.pp.regress_out(adata, ['total_counts', 'pct_counts_mt'])

In [ ]:
#x = np.array(adata.X).flatten()
#plt.hist(x[x>0], bins=20)
#plt.title('Gene dispersions counts')
#plt.xlabel('dispersions')
#plt.ylabel('counts')
#plt.show()

In [ ]:
#sc.pp.scale(adata, max_value=4)

In [ ]:
#x = np.array(adata.X).flatten()
#plt.hist(x[x>0], bins=20)
#plt.title('Gene dispersions counts')
#plt.xlabel('dispersions')
#plt.ylabel('counts')
#plt.show()

In [ ]:
sc.tl.pca(adata, n_comps = 100, svd_solver='arpack')

In [ ]:
sc.pl.pca(adata, color = ["Classification", "treat"])

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100, log=True)

In [ ]:
sc.pl.pca_variance_ratio(adata, n_pcs = 100)

In [ ]:
#adata.write(results_file)
adata

In [ ]:
sc.pp.neighbors(adata, n_neighbors=25, n_pcs=60)

In [ ]:
sc.tl.umap(adata)

In [ ]:
sc.tl.leiden(adata, resolution=0.3)

In [ ]:
sc.pl.umap(adata, color=["treat","leiden","Classification"])

In [ ]:
sc.pl.umap(adata, color=['total_counts', 'pct_counts_mt'])

In [ ]:
sc.pl.umap(adata[adata.obs.treat=="ctl",:], color=["treat","leiden","Classification"])

In [ ]:
sc.pl.umap(adata[adata.obs.treat=="wk1",:], color=["treat","leiden","Classification"])

In [ ]:
sc.pl.umap(adata[adata.obs.treat=="wk4",:], color=["treat","leiden","Classification"])

In [ ]:
adata

In [ ]:
hashID = list(adata.obs.Classification.cat.categories)
hashID

In [ ]:
for i in hashID:
    sc.pl.umap(adata[adata.obs.Classification==i,:], color=["treat","leiden","Classification"])

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')#,use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
gemmMarkers = pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(20)
gemmMarkers

In [ ]:
result = adata.uns['rank_genes_groups']
groups = result['names'].dtype.names
markerStats = pd.DataFrame(
    {group + '_' + key[:1]: result[key][group]
    for group in groups for key in ['names', "scores", 'pvals_adj','logfoldchanges']}).head(100)

In [ ]:
markerStats.iloc[0:20,16:20]

In [ ]:
gemmMarkers.loc[:,"4"]

In [ ]:
sc.pl.umap(adata, color=list(gemmMarkers.loc[:,"4"]), ncols=4)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(100).to_csv("sarcChemoTreat.csv")

0 - Stem, ECM, EMT; FBN1 FAP
1 - ; Skeletal (Peg3+) rhabdo?
2 - NFKb hypoxia-ish, bone marrow stroma, myeliod
3 - apoptosis as a result of dox?
4 - Hypoxia 
5 - ;Fibroblast, FAP (Meg3)
6 - cell mgration; fibroblast FBN1 FAP


In [ ]:
features = ["ECM","Peg3_Skel","Nfkb","Apop","Hypox","Meg3_Fib","Mig"]
sc.tl.leiden(adata, resolution=0.2, key_added = "feature")
adata.rename_categories('feature', features)
sc.pl.umap(adata, color=["feature"])

In [ ]:
sc.tl.dendrogram(adata, groupby = 'leiden')
sc.pl.rank_genes_groups_heatmap(adata)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(100)#.to_csv("write/mouseTreated_HVG.csv")

In [ ]:
sc.tl.rank_genes_groups(adata, 'treat', method='wilcoxon',use_raw=False)
sc.pl.rank_genes_groups(adata, n_genes=25, sharey=False)

In [ ]:
pd.DataFrame(adata.uns['rank_genes_groups']['names']).head(10)

In [ ]:
treatHvg = pd.read_csv("../pdx/analysis/sarc/hvg/treatedHvg.csv")
treatHvg

In [ ]:
mh = pd.read_csv("../scCompare/write/hgncHM_121.csv")#, index_col = 0)
mh

In [ ]:
mouseGenes3 = np.array(treatHvg["2"])
mGene121 = np.array(mh.loc[:,"human"])
for i, mGene in enumerate(treatHvg["2"]):
    if mGene in mGene121:
        hGene = mh.loc[np.where(mGene121 == mGene)[0][0],"mouse"]
        mouseGenes3[i] = hGene#f"{hGene}/{mGene}"
mouseGenes3

mouseGenes7 = np.array(treatHvg["6"])
mGene121 = np.array(mh.loc[:,"human"])
for i, mGene in enumerate(treatHvg["6"]):
    if mGene in mGene121:
        hGene = mh.loc[np.where(mGene121 == mGene)[0][0],"mouse"]
        mouseGenes7[i] = hGene#f"{hGene}/{mGene}"
mouseGenes7

In [ ]:
mouseGenes3

In [ ]:
sc.tl.score_genes(adata, mouseGenes3,score_name='treated_3')
sc.tl.score_genes(adata, mouseGenes7,score_name='treated_7')
sc.pl.umap(adata,color=["treated_3","treated_7","Classification"])

In [ ]:
sc.pl.umap(adata, color=["Myc","Peg3","Meg3","Krt8","Krt8","Akap9","Vegfa","Pdk1"])

In [ ]:
sc.pl.umap(adata, color=["Myc","Peg3","Meg3","Krt8","Krt8","Akap9","Vegfa","Pdk1"])

In [ ]:
adata

In [ ]:
sc.read(resultsFile)

In [ ]:
adata.write('write/scSarcChemo_G1_tum.h5ad')

In [ ]:
adata.obs[["Classification","leiden"]]

In [ ]:
adataT = adata
adataT